# MNIST Classifier using Tensorflow.learn
This is a practice implementation of this [example](https://github.com/amygdala/tensorflow-workshop/blob/master/workshop_sections/mnist_series/02_README_mnist_tflearn.md)

In [1]:
# Import Dependancies
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Declare Variables
DATA_SETS = None
BATCH_SIZE = 10
NUM_STEPS = 400
MODEL_DIR = '/tmp/tfmodels/mnist_tflearn'
DATA_DIR = '/tmp/MNIST_data'

In [3]:
# configure Tensorflow logging
tf.logging.set_verbosity(tf.logging.INFO)

In [4]:
# Define input generator function
def generate_input_fn(dataset, batch_size=BATCH_SIZE):
    def _input_fn():
        # define placeholders
        X = tf.constant(dataset.images)
        Y = tf.constant(dataset.labels, dtype=tf.int32)

        # get a batch of images and labels
        image_batch, label_batch = tf.train.shuffle_batch(
                                [X,Y],
                                batch_size=batch_size,
                                capacity=8*batch_size,
                                min_after_dequeue=4*batch_size,
                                enqueue_many=True)
    
        return {'pixels': image_batch}, label_batch
    return _input_fn

In [5]:
# define and run our linear_classifier
def define_and_run_linear_classifier(num_steps, logdir, batch_size=BATCH_SIZE):
    # Define what our features columns look like
    feature_columns = [tf.contrib.layers.real_valued_column(
        "pixels", dimension=784)]

    # define our linear classifer
    classifier = tf.contrib.learn.LinearClassifier(
        feature_columns=feature_columns,
        n_classes=10,
        model_dir=logdir
    )
    
    # fit our classifier
    classifier.fit(
        input_fn=generate_input_fn(DATA_SETS.train, batch_size=batch_size),
        steps=num_steps
    )

    print("Finished training.")

    # Evaluate accuracy
    accuracy_score = classifier.evaluate(
        input_fn=generate_input_fn(
            DATA_SETS.test,
            batch_size
        ),
        steps=100
    )['accuracy']

    print('Linear Classifier Accuracy: {0:f}'.format(accuracy_score))

In [6]:
# Define and run a Deep Neural Network
def define_and_run_dnn_classifier(num_steps, logdir, lr=.5, batch_size=40):
    # define what the feature columns look like
    feature_columns = [tf.contrib.layers.real_valued_column("pixels", dimension=784)]
    
    # define DNN classifier
    classifier = tf.contrib.learn.DNNClassifier(
        feature_columns=feature_columns,
        n_classes=10,
        hidden_units=[10, 10, 10],
        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=lr),
        model_dir=logdir
    )
    
    # Fit the training data to the  classifier
    classifier.fit(
        input_fn=generate_input_fn(
            DATA_SETS.train,
            batch_size=batch_size
        ),
        steps=num_steps
    )
    
    print("Finished running the deep training via the fit() method")
    
    # Evaluate the DNN classifier's accuracy
    print("\n---Evaluating DNN classifier accuracy...")
    accuracy_score = classifier.evaluate(
        input_fn=generate_input_fn(
            DATA_SETS.test,
            batch_size=batch_size
        ),
        steps=100
    )['accuracy']
    
    print('"DNN Classifier Accuracy: {0:f}'.format(accuracy_score))

In [7]:
# Read in data and download if necessary
global DATA_SETS
print("Downloading and reading data sets...")
DATA_SETS = input_data.read_data_sets(DATA_DIR)

Extracting /tmp/MNIST_data/train-images-idx3-ubyte.gz
Extracting /tmp/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
# # Run Linear Classifier
# print("------Running linear classifier....")
# model_dir = os.path.join(MODEL_DIR, str(int(time.time())))
# define_and_run_linear_classifier(NUM_STEPS, model_dir)

In [9]:
# Run DNN classifier
print("\n-------- Running DNN Classifier...")
define_and_run_dnn_classifier(NUM_STEPS, MODEL_DIR)


-------- Running DNN Classifier...
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb71c36e5c0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create Ch

In [10]:
# Run the follow and go to localhost:6006 for data
# --logdir should === MODEL_DIR
# tensorboard --logdir /tmp/tfmodels/mnist_tflearn